In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
model_dir = '/content/gdrive/My\ Drive/Winter\ 2020/DL/Project\ 2/models/'
!cp /content/gdrive/My\ Drive/Winter\ 2020/DL/Project\ 2/data/*.zip .
!unzip /content/sudoku.zip
!mkdir /content/test
!unzip /content/sudoku_test.zip -d /content/test
!mv /content/test/sudoku.csv /content/sudoku_test.csv

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
Archive:  /content/sudoku.zip
  inflating: sudoku.csv              
Archive:  /content/sudoku_test.zip
  inflating: /content/test/sudoku.csv  


In [3]:
!git clone https://github.com/cloughurd/drl-sudoku.git
!mv drl-sudoku/data/* .
!mv drl-sudoku/models/* .

Cloning into 'drl-sudoku'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 75 (delta 38), reused 34 (delta 11), pack-reused 0
Unpacking objects: 100% (75/75), done.


In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
from torch.utils.data.sampler import SubsetRandomSampler

assert torch.cuda.is_available()
from IPython.core.ultratb import AutoFormattedTB
__ITB__ = AutoFormattedTB(mode = 'Verbose',color_scheme='LightBg', tb_offset = 1)

In [5]:
from cnn import BasicNet
from dataloader import *
from display import *

train_loader = get_loader(root='/content/', batch_size=512, train=True)
test_loader = get_loader(root='/content/', batch_size=1, train=False)
const_x, const_y = next(iter(test_loader))
print(const_x, '\n', const_y)
net = BasicNet(1)
net = net.cuda()
optimizer = optim.Adam(net.parameters())
objective = nn.CrossEntropyLoss()
losses = []

tensor([[[[0., 0., 4., 3., 0., 0., 2., 0., 9.],
          [0., 0., 5., 0., 0., 9., 0., 0., 1.],
          [0., 7., 0., 0., 6., 0., 0., 4., 3.],
          [0., 0., 6., 0., 0., 2., 0., 8., 7.],
          [1., 9., 0., 0., 0., 7., 4., 0., 0.],
          [0., 5., 0., 0., 8., 3., 0., 0., 0.],
          [6., 0., 0., 0., 0., 0., 1., 0., 5.],
          [0., 0., 3., 5., 0., 8., 6., 9., 0.],
          [0., 4., 2., 9., 1., 0., 3., 0., 0.]]]], dtype=torch.float64) 
 tensor([[[7., 5., 3., 2., 6., 0., 1., 4., 8.],
         [2., 1., 4., 7., 3., 8., 6., 5., 0.],
         [8., 6., 0., 1., 5., 4., 7., 3., 2.],
         [3., 2., 5., 0., 8., 1., 4., 7., 6.],
         [0., 8., 7., 5., 4., 6., 3., 2., 1.],
         [1., 4., 6., 3., 7., 2., 8., 0., 5.],
         [5., 7., 8., 6., 2., 3., 0., 1., 4.],
         [6., 0., 2., 4., 1., 7., 5., 8., 3.],
         [4., 3., 1., 8., 0., 5., 2., 6., 7.]]], dtype=torch.float64)


In [0]:
def train(num_epochs=20, print_val_freq=10000, save_freq=10000):
    step = 0
    for i in range(num_epochs):
        loop = tqdm(total=len(train_loader), position=0, leave=False)
        for x, y in train_loader:
            optimizer.zero_grad()
            
            x = x.cuda().float()
            y_hat = net(x)
            
            loss = objective(y_hat.transpose(1,2), y.reshape((-1, 81)).cuda().long())
            loss.backward()
            optimizer.step()

            loop.set_description('epoch:{}, loss:{:.4f}'.format(i, loss.item()))
            loop.update(1)
            
            losses.append(loss.item())
            
            if step % print_val_freq == 0:
                const_y_hat = net(const_x.cuda().float()).detach().cpu()
                print('\n')
                print_tensor_puzzle(const_y_hat[0].reshape(9,9,9))
            if step % save_freq == 0:
                torch.save(model.state_dict(), model_dir + f'bignet-{i}.mod')
            step += 1

In [0]:
train(12)

epoch:0, loss:2.1837:   0%|          | 1/5860 [00:00<42:45,  2.28it/s]

|-------|-------|-------|
| 6 0 0 | 8 6 3 | 5 1 3 | 
| 8 4 0 | 8 8 1 | 1 2 1 | 
| 1 7 3 | 2 0 2 | 2 4 7 | 
|-------|-------|-------|
| 5 0 3 | 6 6 5 | 1 5 8 | 
| 7 8 1 | 3 1 7 | 8 4 6 | 
| 1 3 2 | 4 8 6 | 0 1 2 | 
|-------|-------|-------|
| 7 7 7 | 3 0 6 | 6 6 8 | 
| 1 1 3 | 3 7 7 | 8 5 0 | 
| 3 4 6 | 7 3 0 | 6 0 0 | 
|-------|-------|-------|


epoch:0, loss:0.4976:  17%|█▋        | 1002/5860 [02:55<14:10,  5.71it/s]

|-------|-------|-------|
| 3 0 3 | 2 7 0 | 1 5 8 | 
| 2 2 4 | 7 3 8 | 7 5 0 | 
| 8 7 8 | 1 5 0 | 4 3 2 | 
|-------|-------|-------|
| 3 2 5 | 0 4 1 | 8 7 7 | 
| 0 8 7 | 5 4 6 | 3 2 1 | 
| 3 4 6 | 0 8 2 | 8 0 5 | 
|-------|-------|-------|
| 5 7 8 | 1 2 3 | 0 1 4 | 
| 4 0 2 | 5 3 7 | 6 8 1 | 
| 7 3 1 | 8 0 5 | 2 5 7 | 
|-------|-------|-------|


epoch:0, loss:0.4373:  34%|███▍      | 2002/5860 [05:46<11:07,  5.78it/s]

...

epoch:1, loss:0.3901:  54%|█████▎    | 3142/5860 [09:23<08:02,  5.63it/s]

|-------|-------|-------|
| 7 5 3 | 2 4 4 | 1 5 8 | 
| 2 2 4 | 1 3 8 | 6 6 0 | 
| 8 6 8 | 0 5 0 | 4 3 2 | 
|-------|-------|-------|
| 3 2 5 | 0 4 1 | 4 7 6 | 
| 0 8 7 | 5 4 6 | 3 2 5 | 
| 3 4 6 | 5 7 2 | 8 0 1 | 
|-------|-------|-------|
| 5 7 8 | 1 2 3 | 0 1 4 | 
| 6 0 2 | 4 1 7 | 5 8 3 | 
| 4 3 1 | 8 0 5 | 2 6 7 | 
|-------|-------|-------|


epoch:1, loss:0.3849:  71%|███████   | 4142/5860 [12:22<05:12,  5.50it/s]

|-------|-------|-------|
| 7 0 3 | 2 4 0 | 1 5 8 | 
| 2 2 4 | 3 6 8 | 6 6 0 | 
| 8 6 8 | 0 5 4 | 4 3 2 | 
|-------|-------|-------|
| 3 2 5 | 0 8 1 | 8 7 6 | 
| 0 8 7 | 5 4 6 | 3 2 5 | 
| 3 4 6 | 0 7 2 | 8 0 5 | 
|-------|-------|-------|
| 5 7 8 | 1 2 3 | 0 1 4 | 
| 6 0 2 | 4 1 7 | 5 8 3 | 
| 4 3 1 | 8 0 5 | 2 6 7 | 
|-------|-------|-------|


epoch:1, loss:0.3866:  83%|████████▎ | 4851/5860 [14:29<02:55,  5.76it/s]

In [0]:
753 260 148
214 738 650
860 154 732

325 081 476
087 546 321
146 372 805

578 623 014
602 417 583
431 805 267

In [ ]:
plt.plot(losses)
plt.show()

In [ ]:
import json
json.dump(losses, open(model_dir + 'losses.json', 'w'), indent=4)